# Init & Login

In [1]:
from math import ceil
from random import randrange
from time import sleep
from pprint import pprint

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = None
token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

print( token )

spot = spotipy.Spotify( auth = token )
clear_output( wait = True )
sleep( 2 )
print( "TOKEN OBTAINED" )

TOKEN OBTAINED


# Playlists

In [5]:

playlist = {
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
    'study12' : "0ecSAfnD4CulIVnLt26ukI",
    'study13' : "7K9ucByFRgDuZk8KMHeJkL",
    'zd_Over' : "0v26bHydUxcGC5EbMlkjzG",
    # 'zzzDupe' : "1VPXM7m1by79EdEzDqGsHy",
    # 'cringe'  : "2AAUYlKM1nXKHkZUpUSFbv",
}

dupeDump = "1VPXM7m1by79EdEzDqGsHy"
dupeDict = { 'DupeDump' : dupeDump }
backfill = "0v26bHydUxcGC5EbMlkjzG"


# Playlist Functions

In [9]:

def get_playlist_length( playlist_ID ):
    """ Get the number of total tracks in the playlist """
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    return response['total']
    

def fetch_entire_playlist( playlist_ID ):
    """ Get infodump on all plalist tracks """
    plTracks = []
    trCount  = 0
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    Ntracks = response['total']
    while 1:
        trCount += len(response['items'])
        plTracks.extend( response['items'] )
        
        if trCount >= Ntracks:
            break
    
        response = spot.user_playlist_tracks(
            CLIENT_ID, 
            playlist_ID, 
            fields = 'items,uri,name,id,total', 
            limit  = _RESPONSE_LIMIT,
            offset = trCount
        )
    return plTracks


def create_dupe_removal_jobs( plDict, pause_s = 0.5 ):
    """ Scan each playlist for duplicates within itself and from previous in the sorted playlists, Return removal task list """    
    uniqList = list()
    srtdKeys = sorted( list( plDict.keys() ) )
    dumpList = list()
    plIDlist = list()

    def p_exists_in_prev( itemID, currDex ):
        """ Return True if `itemID` exists in a playlist before `currDex`, Otherwise return False """
        for plSet in uniqList[ :currDex ]:
            if itemID in plSet:
                return True
        return False

    # 1. For every playlist in the dict, do
    for i, plName in enumerate( srtdKeys ):

        print( f"##### Playlist {i+1}: {plName} #####" )
        
        # 2. Fetch playlist and establish a running set
        plID      = plDict[ plName ]
        trkList_i = fetch_entire_playlist( plID )
        trkSet_i  = set([])
        dumpLst_i = list()
        plIDlist.append( plID )
        origLen_i = len( trkList_i )
        print( f"Fetched {origLen_i} tracks!" )
        
        # 3. For every track j in playlist i, do
        j = 0
        while j < len( trkList_i ):

            track_j  = trkList_i[j]
            p_dump_j = False
            
            # 4. Test 1: Did we find this song earlier in the playlist?
            trackID_j = track_j['track']['id']
            if trackID_j in trkSet_i:
                dumpLst_i.append( (trackID_j, j,) )
                p_dump_j = True
                trkList_i.pop(j)

            # 5. Test 2: Did we find this song in and earlier playlist?
            if (not p_dump_j) and p_exists_in_prev( trackID_j, i ):
                dumpLst_i.append( (trackID_j, j,) )
                p_dump_j = True
                trkList_i.pop(j)
            
            # 6. Uniqify
            if not p_dump_j:
                trkSet_i.add( trackID_j )
                j += 1
            
        # 7. Store track set i and dump list
        uniqList.append( trkSet_i )
        dumpList.append( dumpLst_i )
        print( f"In {plName}/{plID}: Retain {len(trkSet_i)}, Dump {len(dumpLst_i)}, Valid? {(len(trkSet_i)+len(dumpLst_i))==origLen_i}\n" )
        sleep( pause_s )

    # N. Return removal job list
    return list( zip( plIDlist, dumpList ) )
            

def run_dupe_removal_jobs( jobList, plTarget = None, pause_s = 0.5 ):
    """ Run jobs created in `create_dupe_removal_jobs` and optionally store them in `plTarget` in the event of a massive fuckup """
    print( f"########## About to run {len(jobList)} jobs ... ##########\n" )
    for i, (plID_i, remLst_i) in enumerate( jobList ):
        print( f"##### Job {i+1}: {plID_i}, {len(remLst_i)} to remove ... #####" )

        if len( remLst_i ):
            remvLs_i = [item[0] for item in remLst_i]

            if plTarget is not None:
                remvLs_ii = remvLs_i[:]
                while len( remvLs_ii ):
                    if len( remvLs_ii ) > _RESPONSE_LIMIT:
                        remvLs_jj = remvLs_ii[:_RESPONSE_LIMIT]
                        remvLs_ii = remvLs_ii[_RESPONSE_LIMIT:]
                    else:
                        remvLs_jj = remvLs_ii[:]
                        remvLs_ii = list()
                    
                    print( f"Venting dupes to {plTarget} ..." )
                    spot.user_playlist_add_tracks( CLIENT_ID, plTarget, remvLs_jj )
                    sleep( pause_s )
    
            print( f"removing dupes from {plID_i} ..." )
            for (trackID, j) in remLst_i:
                
                res = spot.playlist_remove_specific_occurrences_of_items( 
                    plID_i, 
                    [{'uri': trackID, 'positions':[j,]},]
                )
                print( "\tRemove:", trackID, j, res )
                sleep( pause_s )
        print()
    print( f"########## Completed {len(jobList)} jobs! ##########\n" )


def backfill_all_to_limit( plDict, plBckFll, N_limit = 400, pause_s = 0.5 ):
    """ Use `plBckFll` to top off each playlist in `plDict` up to `N_limit` """
    
    srtdKeys = sorted( list( plDict.keys() ) )
    fillBank = fetch_entire_playlist( plBckFll )
    NbkFlBgn = len( fillBank )
    Nfill    = 0

    print( f"\n########## Beginning with a backfill bank of {NbkFlBgn} items! ##########\n" )
    
    # 1. For every playlist in the dict, do
    for i, plName in enumerate( srtdKeys ):
        print( f"##### Playlist {i+1}: {plName} #####" )
        
        # 2. Calc backfill need
        plID = plDict[ plName ]
        plN  = get_playlist_length( plID )
        
        # 3. If backfill needed, perform backfill
        if (plN < N_limit) and (plID != plBckFll):
            Nbf   = N_limit - plN
            if Nbf > len( fillBank ):
                print( f"\tWARN: Needed {Nbf} tracks, but only {len( fillBank )} available! Skipping playlist ...\n" )
                continue
            Nfill += Nbf
            bkLs  = [item['track']['id'] for item in fillBank[ :Nbf ]]

            # 4. Top off playlist
            res = spot.user_playlist_add_tracks( CLIENT_ID, plID, bkLs )
            print( f"\tAdd {len(bkLs)} tracks: {res}" )
            sleep( pause_s )

            # 4. Pop from backfill bank
            for bkfl_j in bkLs:
                res = spot.playlist_remove_specific_occurrences_of_items( 
                    plBckFll, 
                    [{'uri': bkfl_j, 'positions':[0,]},]
                )
                print( "\tRemove:", bkfl_j, res )
                sleep( pause_s )

            fillBank = fillBank[ Nbf: ]
        print()

    NbkFlEnd = get_playlist_length( plBckFll )
    print( f"########## Ending with a backfill bank of {NbkFlEnd} items!, Valid?: {(NbkFlBgn)==(NbkFlEnd+Nfill)} ##########\n" )
                
    
        

# Identify Duplicates

In [10]:
jobList = create_dupe_removal_jobs( playlist )

##### Playlist 1: study01 #####
Fetched 400 tracks!
In study01/0a2qoe6S7lYeZ6nlhZdA0v: Retain 400, Dump 0, Valid? True

##### Playlist 2: study02 #####
Fetched 400 tracks!
In study02/6gbtR2cBq5PvkghidCvvGk: Retain 400, Dump 0, Valid? True

##### Playlist 3: study03 #####
Fetched 400 tracks!
In study03/3o3lN2qntdEV7UKTuuC77K: Retain 400, Dump 0, Valid? True

##### Playlist 4: study04 #####
Fetched 400 tracks!
In study04/41sFSisljvBDMBXtpp5NIw: Retain 400, Dump 0, Valid? True

##### Playlist 5: study05 #####
Fetched 400 tracks!
In study05/02iS5AFGp8YVuUUqcQf8ys: Retain 400, Dump 0, Valid? True

##### Playlist 6: study06 #####
Fetched 400 tracks!
In study06/6KI7A4MWrSM7EyKRUjxIi1: Retain 400, Dump 0, Valid? True

##### Playlist 7: study07 #####
Fetched 400 tracks!
In study07/3V055Md2JdrUT8tX0af7di: Retain 400, Dump 0, Valid? True

##### Playlist 8: study08 #####
Fetched 400 tracks!
In study08/0tspdJlwSgiyf2O9PO6QaP: Retain 400, Dump 0, Valid? True

##### Playlist 9: study09 #####
Fetched 

# Move Duplicates

In [11]:
run_dupe_removal_jobs( jobList, dupeDump, pause_s = 0.5 )

########## About to run 15 jobs ... ##########

##### Job 1: 0a2qoe6S7lYeZ6nlhZdA0v, 0 to remove ... #####

##### Job 2: 6gbtR2cBq5PvkghidCvvGk, 0 to remove ... #####

##### Job 3: 3o3lN2qntdEV7UKTuuC77K, 0 to remove ... #####

##### Job 4: 41sFSisljvBDMBXtpp5NIw, 0 to remove ... #####

##### Job 5: 02iS5AFGp8YVuUUqcQf8ys, 0 to remove ... #####

##### Job 6: 6KI7A4MWrSM7EyKRUjxIi1, 0 to remove ... #####

##### Job 7: 3V055Md2JdrUT8tX0af7di, 0 to remove ... #####

##### Job 8: 0tspdJlwSgiyf2O9PO6QaP, 0 to remove ... #####

##### Job 9: 5mHRBFoQtYy2izeZ66pG95, 0 to remove ... #####

##### Job 10: 3832xeKGEOAXFJqE4K8kIq, 0 to remove ... #####

##### Job 11: 65MXR4dubPL9t0P4dgTWvn, 0 to remove ... #####

##### Job 12: 0ecSAfnD4CulIVnLt26ukI, 0 to remove ... #####

##### Job 13: 7K9ucByFRgDuZk8KMHeJkL, 0 to remove ... #####

##### Job 14: 0v26bHydUxcGC5EbMlkjzG, 0 to remove ... #####

##### Job 15: 1VPXM7m1by79EdEzDqGsHy, 432 to remove ... #####
Venting dupes to 5TtKaKCouyJp7Hhtu4YlYm ...
V

# Backfill

In [4]:
backfill_all_to_limit( playlist, backfill, N_limit = 400, pause_s = 0.5 )


########## Beginning with a backfill bank of 281 items! ##########

##### Playlist 1: study01 #####

##### Playlist 2: study02 #####
	Add 1 tracks: {'snapshot_id': 'AAACalZrvTXMggoUN+WtGCak6urjJSVK'}
	Remove: 1NrOCXg2jjWhPdR9mypBMk {'snapshot_id': 'AAAAU4hrYks2gP8D8RNM9V0HY6mgSZJN'}

##### Playlist 3: study03 #####

##### Playlist 4: study04 #####

##### Playlist 5: study05 #####
	Add 1 tracks: {'snapshot_id': 'AAAFuMu/1NgYRYGms7/KXRUgcUWsEe2u'}
	Remove: 0508HxA8xTBbQcRa1Irk6Z {'snapshot_id': 'AAAAVGMLagoroK80RRxQNl6YeQQ3T4YT'}

##### Playlist 6: study06 #####

##### Playlist 7: study07 #####

##### Playlist 8: study08 #####

##### Playlist 9: study09 #####

##### Playlist 10: study10 #####

##### Playlist 11: study11 #####

##### Playlist 12: study12 #####

##### Playlist 13: study13 #####

##### Playlist 14: zd_Over #####


########## Ending with a backfill bank of 279 items!, Valid?: True ##########



# Clean Dupes

In [6]:
dupeCleanJob = create_dupe_removal_jobs( dupeDict )

##### Playlist 1: DupeDump #####
Fetched 471 tracks!
In DupeDump/1VPXM7m1by79EdEzDqGsHy: Retain 471, Dump 0, Valid? True



In [5]:
run_dupe_removal_jobs( dupeCleanJob, None, pause_s = 0.5 )

########## About to run 1 jobs ... ##########

##### Job 1: 1VPXM7m1by79EdEzDqGsHy, 73 to remove ... #####
removing dupes from 1VPXM7m1by79EdEzDqGsHy ...
	Remove: 2CLOq3wn9XwqxxQiTy2uDG 34 {'snapshot_id': 'AAAAGfX3NzZo8KzWtiwAzV0A3whtv1Db'}
	Remove: 3VKcBdlqb1iPtBJIS6Dzqy 37 {'snapshot_id': 'AAAAGprFx3dGRsR+/9rCoChLqKfhV+g+'}
	Remove: 1gO6bqT43JiO6tlzlOGddy 40 {'snapshot_id': 'AAAAG7Zbyjx2zYrvuLBf7ppNtsfvk90v'}
	Remove: 3HOOp13jnPU0Ze1CDV6sXI 49 {'snapshot_id': 'AAAAHH3l0ufrrqulooBksVnF5kkGhxm6'}
	Remove: 0P9mbDZotTY0fLg905oYE5 51 {'snapshot_id': 'AAAAHQvFb33KaDxOpfDpHypUv9/BVzOl'}
	Remove: 2q5cjdEArOwO3Oj8GdV4lJ 67 {'snapshot_id': 'AAAAHgzjbB3MqAAQqY8b5iO76sSYgXWS'}
	Remove: 4RYaufuta8LzkT0EXftcd0 78 {'snapshot_id': 'AAAAH9nbPuCaPvaYU921IbsfC0Fwla8h'}
	Remove: 1M04RE8vg8M2vsVnVaTJBN 99 {'snapshot_id': 'AAAAICQr5QtSaIZKkaVchvyrXRbpNfur'}
	Remove: 0chjZEnr4D28udxQ1D4XxM 113 {'snapshot_id': 'AAAAIRW50TnH0hqm4Ounkei0H5cablwk'}
	Remove: 1SfOvU5O39Dq91IUU6AYlH 118 {'snapshot_id': 'AAAAIvABo